In [ ]:
import csv
import json
from functools import reduce

# Read CSV file and convert to list of dictionaries
file_path = '/content/911_Calls_for_Service_(Last_30_Days).csv'

with open(file_path, mode='r', encoding='utf-8-sig') as file:
    csv_reader = csv.DictReader(file)
    data = [row for row in csv_reader]

# Filter out rows with missing 'Zip' or 'Neighborhood'
filtered_data = list(filter(lambda x: x['zip_code'] and x['neighborhood'], data))

# Helper function to convert time strings to float
def convert_time_to_float(time_str):
    try:
        return float(time_str)
    except ValueError:
        return 0.0

# Calculate total average response time, dispatch time, and total time for Detroit
total_response_time = reduce(lambda acc, x: acc + convert_time_to_float(x['totalresponsetime']), filtered_data, 0.0)
total_dispatch_time = reduce(lambda acc, x: acc + convert_time_to_float(x['dispatchtime']), filtered_data, 0.0)
total_time = reduce(lambda acc, x: acc + convert_time_to_float(x['totaltime']), filtered_data, 0.0)

avg_response_time = total_response_time / len(filtered_data)
avg_dispatch_time = total_dispatch_time / len(filtered_data)
avg_total_time = total_time / len(filtered_data)

# Model neighborhood samples
neighborhoods = set(row['neighborhood'] for row in filtered_data)

neighborhood_samples = []

for neighborhood in neighborhoods:
    neighborhood_data = list(filter(lambda x: x['neighborhood'] == neighborhood, filtered_data))

    total_response_time_neigh = reduce(lambda acc, x: acc + convert_time_to_float(x['totalresponsetime']), neighborhood_data, 0.0)
    total_dispatch_time_neigh = reduce(lambda acc, x: acc + convert_time_to_float(x['dispatchtime']), neighborhood_data, 0.0)
    total_time_neigh = reduce(lambda acc, x: acc + convert_time_to_float(x['totaltime']), neighborhood_data, 0.0)

    avg_response_time_neigh = total_response_time_neigh / len(neighborhood_data)
    avg_dispatch_time_neigh = total_dispatch_time_neigh / len(neighborhood_data)
    avg_total_time_neigh = total_time_neigh / len(neighborhood_data)

    neighborhood_samples.append({
        'neighborhood': neighborhood,
        'AvgResponseTime': avg_response_time_neigh,
        'AvgDispatchTime': avg_dispatch_time_neigh,
        'AvgTotalTime': avg_total_time_neigh
    })

# Add the overall Detroit data
detroit_data = {
    'neighborhood': 'All Detroit',
    'AvgResponseTime': avg_response_time,
    'AvgDispatchTime': avg_dispatch_time,
    'AvgTotalTime': avg_total_time
}

neighborhood_samples.append(detroit_data)

# Write output to JSON
output_file = '/content/detroit_police_report.json'

with open(output_file, mode='w') as json_file:
    json.dump(neighborhood_samples, json_file, indent=4)

print(f'Output written to {output_file}')


Output written to /content/detroit_police_report.json
